## Constants

In [8]:
import sys, os
import pandas as pd
import polars as pl
import numpy as np
import subprocess
import gc
import optuna
from datetime import datetime, timezone
import warnings
import xgboost as xgb
import joblib as jl
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import matthews_corrcoef
from mlflow.models import infer_signature
import mlflow
import random
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold

today = datetime.now(timezone.utc).strftime("%Y_%m_%d")
warnings.filterwarnings("ignore")

from hyper_params import (
    mushroom_tuning_2024_08_06_1722934727_params,
)

# helpers


SEED = 108
random.seed(SEED)
N_FOLDS = 7
# data
train_path = "../../data/mushrooms/train.csv"
test_path = "../../data/mushrooms/test.csv"
cache_path = "../../data/mushrooms/cache"
# model
is_tunning = True
try:
    rs = subprocess.check_output("nvidia-smi")
    device = "cuda" if rs is not None else "cpu"
except (
    Exception
):  # this command not being found can raise quite a few different errors depending on the configuration
    print("No Nvidia GPU in system!")
    device = "cpu"
goal = "binary:logistic"

# custom metric
objective_dict = {
    "binary:logistic": {
        "metric": {
            "is_custom": False,
            "name": "logloss",
            "fval": None,
        },
        "direction": "minimize",
    }
}
# objective_dict = {
#     "binary:logistic": {
#         "metric": {
#             "is_custom": True,
#             "name": "MCC",
#             "fval": mcc_metric_v2,
#         },
#         "direction": "maximize",
#     }
# }
metric = objective_dict[goal]["metric"]["name"]
is_custom_metric = objective_dict[goal]["metric"]["is_custom"]
fval = objective_dict[goal]["metric"]["fval"]
direction = objective_dict[goal]["direction"]

best_params = {
    "device": device,
    "verbosity": 0,
    "objective": goal,
}
best_params.update(mushroom_tuning_2024_08_06_1722934727_params)
best_params

{'device': 'cuda',
 'verbosity': 0,
 'objective': 'binary:logistic',
 'tree_method': 'hist',
 'eta': 0.0696294726051571,
 'max_depth': 0,
 'min_child_weight': 1,
 'gamma': 0.044230646284796976,
 'subsample': 0.9405269471473167,
 'colsample_bytree': 0.2999355523666192,
 'lambda': 0.9746051811186938,
 'alpha': 4.210861941737071}

## Prepare data

In [2]:
train = pd.read_csv(train_path)
print(f"train size: {train.shape}")
X_test = pd.read_csv(test_path)
print(f"test size: {X_test.shape}")
submit_df = pd.DataFrame()
submit_df["id"] = X_test["id"]
y_train = train["class"]
X_train = train.drop(columns=["id", "class"], axis=1)
X_test.drop(columns=["id"], inplace=True, axis=1)

train size: (3116945, 22)
test size: (2077964, 21)


In [3]:
target = "class"

categorical_cols = (
    train.drop(columns=target).select_dtypes(include="object").columns.to_list()
)
# for c in categorical_cols:
#     train[c] = train[c].astype("category")
#     X_test[c] = X_test[c].astype("category")
numerical_cols = (
    train.drop(columns="id").select_dtypes(include="number").columns.to_list()
)
gc.collect()

125

## Data preprocessing

In [4]:
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder

# get top 10 most frequent names
n = 10
for c in categorical_cols:
    train_mode_values = X_train[c].value_counts()[:n].index.tolist()
    X_train.loc[~X_train[c].isin(train_mode_values), c] = "other"
    test_mode_values = X_test[c].value_counts()[:n].index.tolist()
    X_test.loc[~X_test[c].isin(test_mode_values), c] = "other"


# Create the numerical and categorical pipelines
numerical_pipeline = Pipeline(
    steps=[
        ("num_imputer", SimpleImputer(strategy="median")),
        # ("minmax", MinMaxScaler()),
        ("scaler", StandardScaler()),
    ]
)

categorical_pipeline = Pipeline(
    steps=[
        ("cat_imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
        # ("ordinal", OrdinalEncoder()),
    ]
)

# Combine the pipelines into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_pipeline, numerical_cols),
        ("cat", categorical_pipeline, categorical_cols),
    ]
)

# Create the full pipeline with the XGBoost model
data_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
    ]
)
data_pipeline.set_output(transform="polars")
# Preprocess the data
X_all = pd.concat([X_train, X_test])
data_pipeline.fit(pl.from_pandas(X_all))
X_test_transformed = data_pipeline.transform(X_test)
X_train_transformed = data_pipeline.transform(X_train)
# X_val_transformed = preprocessor.transform(X_val)
# Binarize the target labels
lb = LabelBinarizer()

y_train_binarized = lb.fit_transform(y_train)
# y_val_binarized = lb.transform(y_val)

# prepare data for training
# dtrain = xgb.DMatrix(X_train_transformed, label=y_train_binarized)
# # dval = xgb.DMatrix(X_val_transformed, label=y_val_binarized)
# dtest = xgb.DMatrix(X_test_transformed)

gc.collect()

120

## CV

In [5]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


clf: xgb.XGBClassifier = xgb.XGBClassifier(
    **best_params,
    n_estimators=4000,
    early_stopping_rounds=50,
    enable_categorical=True,
)

In [6]:
from tqdm import tqdm

skf = StratifiedKFold(n_splits=N_FOLDS)

# kf = KFold(n_splits=N_FOLDS)
y_preds = []
y_trues = []
for train_index, test_index in tqdm(skf.split(X_train_transformed, y_train_binarized)):
    X_train, X_test = X_train_transformed[train_index], X_train_transformed[test_index]
    y_train, y_test = y_train_binarized[train_index], y_train_binarized[test_index]

    clf.fit(X=X_train, y=y_train, eval_set=[(X_test, y_test)])

    y_pred = clf.predict(X_test)
    y_preds.append(y_pred)
    y_trues.append(y_test)
# Concatenate the predictions and true labels
y_preds_concat = np.concatenate(y_preds)
y_trues_concat = np.concatenate(y_trues)
mcc = matthews_corrcoef(y_trues_concat, y_preds_concat)
print(f"Validation mcc score: {mcc}")

0it [00:00, ?it/s]

[0]	validation_0-logloss:0.65338
[1]	validation_0-logloss:0.59601
[2]	validation_0-logloss:0.55447
[3]	validation_0-logloss:0.52366
[4]	validation_0-logloss:0.49535
[5]	validation_0-logloss:0.45645
[6]	validation_0-logloss:0.42516
[7]	validation_0-logloss:0.39466
[8]	validation_0-logloss:0.36703
[9]	validation_0-logloss:0.34225
[10]	validation_0-logloss:0.32834
[11]	validation_0-logloss:0.31221
[12]	validation_0-logloss:0.29325
[13]	validation_0-logloss:0.28065
[14]	validation_0-logloss:0.26610
[15]	validation_0-logloss:0.24979
[16]	validation_0-logloss:0.23411
[17]	validation_0-logloss:0.22315
[18]	validation_0-logloss:0.21362
[19]	validation_0-logloss:0.20090
[20]	validation_0-logloss:0.18921
[21]	validation_0-logloss:0.18065
[22]	validation_0-logloss:0.16992
[23]	validation_0-logloss:0.16101
[24]	validation_0-logloss:0.15348
[25]	validation_0-logloss:0.14533
[26]	validation_0-logloss:0.14010
[27]	validation_0-logloss:0.13429
[28]	validation_0-logloss:0.12900
[29]	validation_0-loglos

1it [00:18, 18.56s/it]

[0]	validation_0-logloss:0.65340
[1]	validation_0-logloss:0.59600
[2]	validation_0-logloss:0.55441
[3]	validation_0-logloss:0.52353
[4]	validation_0-logloss:0.49524
[5]	validation_0-logloss:0.45637
[6]	validation_0-logloss:0.42505
[7]	validation_0-logloss:0.39453
[8]	validation_0-logloss:0.36714
[9]	validation_0-logloss:0.34230
[10]	validation_0-logloss:0.32840
[11]	validation_0-logloss:0.31226
[12]	validation_0-logloss:0.29328
[13]	validation_0-logloss:0.28068
[14]	validation_0-logloss:0.26615
[15]	validation_0-logloss:0.24987
[16]	validation_0-logloss:0.23421
[17]	validation_0-logloss:0.22325
[18]	validation_0-logloss:0.21371
[19]	validation_0-logloss:0.20099
[20]	validation_0-logloss:0.18930
[21]	validation_0-logloss:0.18076
[22]	validation_0-logloss:0.17003
[23]	validation_0-logloss:0.16113
[24]	validation_0-logloss:0.15361
[25]	validation_0-logloss:0.14546
[26]	validation_0-logloss:0.14021
[27]	validation_0-logloss:0.13414
[28]	validation_0-logloss:0.12883
[29]	validation_0-loglos

2it [00:35, 17.67s/it]

...

[0]	validation_0-logloss:0.65342
[1]	validation_0-logloss:0.59610
[2]	validation_0-logloss:0.55455
[3]	validation_0-logloss:0.52372
[4]	validation_0-logloss:0.49538
[5]	validation_0-logloss:0.45646
[6]	validation_0-logloss:0.42515
[7]	validation_0-logloss:0.39461
[8]	validation_0-logloss:0.36697
[9]	validation_0-logloss:0.34212
[10]	validation_0-logloss:0.32820
[11]	validation_0-logloss:0.31208
[12]	validation_0-logloss:0.29312
[13]	validation_0-logloss:0.28051
[14]	validation_0-logloss:0.26595
[15]	validation_0-logloss:0.24964
[16]	validation_0-logloss:0.23398
[17]	validation_0-logloss:0.22298
[18]	validation_0-logloss:0.21342
[19]	validation_0-logloss:0.20072
[20]	validation_0-logloss:0.18904
[21]	validation_0-logloss:0.18047
[22]	validation_0-logloss:0.16978
[23]	validation_0-logloss:0.16088
[24]	validation_0-logloss:0.15336
[25]	validation_0-logloss:0.14525
[26]	validation_0-logloss:0.14002
[27]	validation_0-logloss:0.13393
[28]	validation_0-logloss:0.12863
[29]	validation_0-loglos

6it [01:43, 17.08s/it]

[0]	validation_0-logloss:0.65337
[1]	validation_0-logloss:0.59598
[2]	validation_0-logloss:0.55439
[3]	validation_0-logloss:0.52359
[4]	validation_0-logloss:0.49528
[5]	validation_0-logloss:0.45639
[6]	validation_0-logloss:0.42507
[7]	validation_0-logloss:0.39453
[8]	validation_0-logloss:0.36690
[9]	validation_0-logloss:0.34210
[10]	validation_0-logloss:0.32819
[11]	validation_0-logloss:0.31207
[12]	validation_0-logloss:0.29312
[13]	validation_0-logloss:0.28053
[14]	validation_0-logloss:0.26597
[15]	validation_0-logloss:0.24968
[16]	validation_0-logloss:0.23403
[17]	validation_0-logloss:0.22305
[18]	validation_0-logloss:0.21351
[19]	validation_0-logloss:0.20081
[20]	validation_0-logloss:0.18913
[21]	validation_0-logloss:0.18057
[22]	validation_0-logloss:0.16984
[23]	validation_0-logloss:0.16095
[24]	validation_0-logloss:0.15343
[25]	validation_0-logloss:0.14530
[26]	validation_0-logloss:0.14007
[27]	validation_0-logloss:0.13400
[28]	validation_0-logloss:0.12872
[29]	validation_0-loglos

7it [02:01, 17.30s/it]


Validation mcc score: 0.9847525278751144


In [7]:
y_preds = clf.predict(X_test_transformed)
pred_classes = lb.inverse_transform(y_preds)
submit_df["class"] = pred_classes
submit_df.to_csv("submission.csv", index=False)